In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

In [1]:
!pip install gensim
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 8.5 MB/s eta 0:00:00


KeyboardInterrupt: 

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
  tokens = text.lower().split()
  tokens = [ word for word in tokens if word.isalpha() and word not in stop_words]
  return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['tokens'] = df['message'].apply(preprocess_text)
print(df)

      label                                            message  \
0         0  Go until jurong point, crazy.. Available only ...   
1         0                      Ok lar... Joking wif u oni...   
2         1  Free entry in 2 a wkly comp to win FA Cup fina...   
3         0  U dun say so early hor... U c already then say...   
4         0  Nah I don't think he goes to usf, he lives aro...   
...     ...                                                ...   
5567      1  This is the 2nd time we have tried 2 contact u...   
5568      0              Will Ì_ b going to esplanade fr home?   
5569      0  Pity, * was in mood for that. So...any other s...   
5570      0  The guy did some bitching but I acted like i'd...   
5571      0                         Rofl. Its true to its name   

                                                 tokens  
0     [go, jurong, available, bugis, n, great, world...  
1                                  [ok, joking, wif, u]  
2     [free, entry, wkly, comp, w

In [ ]:
import gensim.downloader as api
w2v_model = api.load("glove-wiki-gigaword-100")

def vectorize(tokens):
  valid_tokens = [token for token in tokens if token in w2v_model]
  if not valid_tokens:
    return np.zeros(w2v_model.vector_size)
  vectors = [w2v_model[token] for token in valid_tokens]
  return np.mean(vectors, axis=0)

df['vector'] = df['tokens'].apply(vectorize)


In [ ]:
X = np.vstack(df['vector'].values)
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [ ]:
logModel = LogisticRegression(max_iter=1000)
logModel.fit(X_train, y_train)
y_pred = logModel.predict(X_test)

In [ ]:
def predict_message_class(model, w2v_model, message):
    tokens = preprocess_text(message)
    vector = vectorize(tokens).reshape(1, -1)
    prediction = model.predict(vector)[0]
    return "spam" if prediction == 1 else "ham"

msg = "Assignment pending due in 1 hour"
print(f"Message: {msg}")
print("Prediction:", predict_message_class(logModel, w2v_model, msg))




Message: Assignment pending due in 1 hour
Prediction: ham


In [ ]:
msg1 = 'Win 10Rs in 1 min'
print(f"Message: {msg1}")
print("Prediction:", predict_message_class(logModel, w2v_model, msg1))

Message: Win 10Rs in 1 min
Prediction: spam


# Question 2

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
import nltk
import contractions
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

data = pd.read_csv('Tweets.csv', encoding='latin-1')

In [2]:
print(data)

                 tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
...                   ...               ...                           ...   
14635  569587686496825344          positive                        0.3487   
14636  569587371693355008          negative                        1.0000   
14637  569587242672398336           neutral                        1.0000   
14638  569587188687634433          negative                        1.0000   
14639  569587140490866689           neutral                        0.6771   

               negativereason  negativereason_confidence         airline  \

In [6]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [7]:
lemmatizer = nltk.WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {'J': wordnet.ADJ, 'N': wordnet.NOUN,
                'V': wordnet.VERB, 'R': wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def preprocess_tweet(tweet):
  tweet = tweet.lower()
  tweet=contractions.fix(tweet)


  tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)  # URLs
  tweet = re.sub(r'@\w+', '', tweet)                    # Mentions
  tweet = re.sub(r'#\w+', '', tweet)                    # Hashtags
  tweet = re.sub(r'[^\w\s]', '', tweet)                 # Punctuation

  tokens = tweet.split()

  lemmatized = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
  return lemmatized

In [8]:
# Preproseesing tweets
data['tokens'] = data['text'].apply(preprocess_tweet)

In [9]:
print(data)

                 tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
...                   ...               ...                           ...   
14635  569587686496825344          positive                        0.3487   
14636  569587371693355008          negative                        1.0000   
14637  569587242672398336           neutral                        1.0000   
14638  569587188687634433          negative                        1.0000   
14639  569587140490866689           neutral                        0.6771   

               negativereason  negativereason_confidence         airline  \

In [10]:
import gensim.downloader as ge
w2v_model = ge.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [12]:

def vectorize(tokens):
  valid_tokens = [token for token in tokens if token in w2v_model]
  if not valid_tokens:
    return np.zeros(w2v_model.vector_size)
  vectors = [w2v_model[token] for token in valid_tokens]
  return np.mean(vectors, axis=0)

data['vector'] = data['tokens'].apply(vectorize)

In [27]:
X = np.vstack(data['vector'].values)
y = data['airline_sentiment'].values
y = np.where(y == 'neutral', 0, y)
y = np.where(y == 'positive', 1, y)
y = np.where(y == 'negative', 2, y)
y = y.astype(int) # Convert to integer type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
print(y)

[0 1 0 ... 0 2 0]


In [28]:
logReg = LogisticRegression(max_iter=2000)
logReg.fit(X_train, y_train)
y_pred = logReg.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")

Accuracy: 0.7540983606557377
Recall: 0.7540983606557377
Precision: 0.7400951674698802
F1 Score: 0.7303688825291269


In [30]:
def predict_tweet_senti(model, w2v_model, tweet):
    tokens = preprocess_tweet(tweet)
    vector = vectorize(tokens).reshape(1, -1)
    prediction = model.predict(vector)[0]
    return "neutral" if prediction == 0 else "positive" if prediction == 1 else "negative"

In [31]:
twt1 = 'Uncomfortable experience, bad service'
print(f"Tweet: {twt1}")
print("Prediction:", predict_tweet_senti(logReg, w2v_model, twt1))

Tweet: Uncomfortable experience, bad service
Prediction: negative


In [34]:
twt2= 'Good Price for the experience'
print(f"Tweet: {twt2}")
print("Prediction:", predict_tweet_senti(logReg, w2v_model, twt2))

Tweet: Good Price for the experience
Prediction: positive


In [35]:
twt3 = '@Indigo My Name is Ayush'
print(f"Tweet: {twt3}")
print("Prediction:", predict_tweet_senti(logReg, w2v_model, twt3))

Tweet: @Indigo My Name is Ayush
Prediction: neutral
